In [1]:
# CSE6242/CX4242 Homework 4 Sketch Code
# Please use this outline to implement your decision tree. You can add any code around this.

import csv
import pandas as pd
from math import log
from graphviz import Digraph

In [2]:
with open('ex6Data.csv') as f:
#     next(f,None)
    data = [tuple(line) for line in csv.reader(f, delimiter=",")]
print("Number of records: %d" % len(data))

Number of records: 4899


In [3]:
data = pd.DataFrame(data[1:], columns=data[0])
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8,0
1,6.3,0.3,0.34,1.6,0.049,14,132,0.994,3.3,0.49,9.5,0
2,8.1,0.28,0.4,6.9,0.05,30,97,0.9951,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,0


In [4]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898,4898,4898,4898,4898,4898,4898,4898,4898,4898,4898,4898
unique,68,125,87,310,160,132,251,890,103,79,104,2
top,6.8,0.28,0.3,1.2,0.044,29,111,0.992,3.14,0.5,9.4,0
freq,308,263,307,187,201,160,69,64,172,249,229,3838


In [5]:
data = data.astype(float)

In [6]:
# Implement your decision tree below
class DecisionTree():
    def learn(self, training_set, attr_names, threshold, method='InformationGain'):
        # implement this function
        attr_set = set(range(len(training_set[0])-1))
        self.attr_list = [str(x) for x in attr_names]
        self.threshold = threshold
        self.method = 'InformationGain'
        self.tree = self.__treeGrowth(training_set, attr_set)
            
    def __treeGrowth(self, training_set, attr_set):
        labels = [row[-1] for row in training_set]
        if len(set(labels)) == 1 or len(training_set) <= self.threshold:
            label = max(labels, key=labels.count)
            sub_tree = {'type':'leaf', 'label':label}
            return sub_tree
        else:
            sub_tree = {'type':'node'}
            # 计算每个属性最好的分裂点的分数和分裂点
            attr_scores_split = [self.__eval_attr(training_set, attr_idx, method=self.method) for attr_idx in attr_set]
            attr_scores = [x[0] for x in attr_scores_split]
            best_attr_idx = attr_scores.index(max(attr_scores))
            best_attr_split_point = attr_scores_split[best_attr_idx][1]
            # 存储属性和分裂点
            sub_tree['test_condition'] = (best_attr_idx, best_attr_split_point)
            # 根据选取属性的分裂点分裂数据
            right_training_set = [row for row in training_set if row[best_attr_idx] > best_attr_split_point]
            left_training_set = [row for row in training_set if row[best_attr_idx] <= best_attr_split_point]
#             attr_set = attr_set.remove(best_attr_idx)
            sub_tree['left_sub_tree'] = self.__treeGrowth(left_training_set, attr_set)
            sub_tree['right_sub_tree'] = self.__treeGrowth(right_training_set, attr_set)
            return sub_tree
        
    # implement this function
    def classify(self, test_instance):
        result = self.__test(test_instance, self.tree)
        return result
    
    def visualize(self):
        self.node_names = [0]
        self.g = Digraph("G", filename='tree', format='png', strict=False)
        if self.tree['type'] == 'leaf':
            node_name = str(self.node_names[-1])
            self.node_names.append(self.node_names[-1]+1)
            node_label = self.attr_list[-1]+': '+str(self.tree['label'])
            self.g.node(name=node_name, label=node_label)
            self.g.view()
        else:
            test_attr_idx, split_point = self.tree['test_condition']
            root_name = str(self.node_names[-1])
            self.node_names.append(self.node_names[-1]+1)
            root_label = self.attr_list[test_attr_idx]+' > '+str(split_point)+' ?'
            self.g.node(name=root_name,label=root_label)
            
            left_child_name = self.__vis_subtree(self.tree['left_sub_tree'])
            self.g.edge(root_name,left_child_name,label='no')
            right_child_name = self.__vis_subtree(self.tree['right_sub_tree'])
            self.g.edge(root_name,right_child_name,label='yes')
            
            self.g.view()
    
    def __vis_subtree(self, tree):
        # 画出tree并返回根节点的name
        if tree['type'] == 'leaf':
            node_name = str(self.node_names[-1])
            self.node_names.append(self.node_names[-1]+1)
            node_label = self.attr_list[-1]+': '+str(tree['label'])
            self.g.node(name=node_name, label=node_label)
            return node_name
        else:
            test_attr_idx, split_point = tree['test_condition']
            root_name = str(self.node_names[-1])
            self.node_names.append(self.node_names[-1]+1)
            root_label = self.attr_list[test_attr_idx]+' > '+str(split_point)+' ?'
            self.g.node(name=root_name,label=root_label)
            
            left_child_name = self.__vis_subtree(tree['left_sub_tree'])
            self.g.edge(root_name,left_child_name,label='no')
            right_child_name = self.__vis_subtree(tree['right_sub_tree'])
            self.g.edge(root_name,right_child_name,label='yes')
            
            return root_name
    
    def __test(self, test_instance, tree):
        if tree['type'] == 'leaf':
            return tree['label']
        else:
            test_attr, split_point = tree['test_condition']
            if test_instance[test_attr] <= split_point:
                return self.__test(test_instance, tree['left_sub_tree'])
            else:
                return self.__test(test_instance, tree['right_sub_tree'])
    
    def __eval_attr(self, training_set, attr_idx, method='InformationGain'):
        if method == 'InformationGain':
            return self.__InformationGain(training_set, attr_idx)
        if method == 'GainRatio':
            return self.__GainRatio(training_set, attr_idx)
        if method == 'GiniGain':
            return self.__GiniGain(training_set, attr_idx)
        if method == 'MissclassificationError':
            return self.__MissclassificationError(training_set, attr_idx)
        
    # 以下方法实现评估属性，返回给定属性的最佳分裂点及分数
    def __InformationGain(self, training_set, attr_idx):
        x = [row[attr_idx] for row in training_set]
        y = [row[-1] for row in training_set]
        attr_values = list(set(x))
        best_split = -1
        least_entropy = 1
        for i in range(len(attr_values)-1):
            split = (attr_values[i]+attr_values[i+1])/2
            x1_idx = [i for i in range(len(x)) if x[i] <= split]
            x2_idx = [i for i in range(len(x)) if x[i] > split]
            y1 = [y[i] for i in x1_idx]
            y2 = [y[i] for i in x2_idx]
            entropy1, entropy2 = self.__entropy(y1), self.__entropy(y2)
            average_entropy = entropy1*len(y1)/len(y)+entropy2*len(y2)/len(y)
            if average_entropy < least_entropy:
                least_entropy = average_entropy
                best_split = split
        entropy = self.__entropy(y)
        IG = entropy - least_entropy
        return (IG, best_split)
    
    def __entropy(self, labels):
        label_values = set(labels)
        totol_num = len(labels)
        entropy = 0
        for label_value in label_values:
            p = labels.count(label_value)/totol_num
            entropy -= p*log(p,2)
        return entropy
    
    def __GainRatio(self, training_set, attr_idx):
        attr_vals = [row[attr_idx] for row in training_set]
        val_count = {}
        for val in attr_vals:
            if val in val_count:
                val_count[val] += 1
            else:
                val_count[val] = 1
        train_len = len(attr_vals)
        intrinsic = 0
        for val in val_count:
            ratio = val_count[val]/train_len
            intrinsic -= ratio*log(ratio)
        IG, best_split = self.__InformationGain
        return (IG/intrinsic, best_split)

    def __GiniGain(self, training_set, attr_idx):
        x = [row[attr_idx] for row in training_set]
        y = [row[-1] for row in training_set]
        attr_values = list(set(x))
        best_split = -1
        least_gini = 1
        for i in range(len(attr_values)-1):
            split = (attr_values[i]+attr_values[i+1])/2
            x1_idx = [i for i in range(len(x)) if x[i] <= split]
            x2_idx = [i for i in range(len(x)) if x[i] > split]
            y1 = [y[i] for i in x1_idx]
            y2 = [y[i] for i in x2_idx]
            gini1, gini2 = self.__gini(y1), self.__gini(y2)
            average_gini = gini1*len(y1)/len(y)+gini2*len(y2)/len(y)
            if average_gini < least_gini:
                least_gini = average_gini
                best_split = split
        gini0 = self.__gini(y)
        gini_gain = gini0 - gini
        return (gini_gain, best_split)
    
    def __gini(self, labels):
        label_values = set(labels)
        totol_num = len(labels)
        gini = 1
        for label_value in label_values:
            p = labels.count(label_value)/totol_num
            gini -= p*p
        return gini
    
    def __MissclassificationError(self, training_set, attr_idx):
        pass

In [7]:
def run_decision_tree(threshold=1, visualize=False):

    # Load data set
    with open("ex6Data.csv") as f:
        lines = csv.reader(f, delimiter=',')
        data = [[float(x) for x in line] if idx != 0 else [x for x in line] 
                for idx,line in enumerate(csv.reader(f, delimiter=","))]
        attr_names, data = data[0], data[1:]
    print("Number of records: %d" % len(data))

    # Split training/test sets
    # You need to modify the following code for cross validation.
    K = 10
    training_set = [x for i, x in enumerate(data) if i % K != 9]
    test_set = [x for i, x in enumerate(data) if i % K == 9]
    
    tree = DecisionTree()
    # Construct a tree using training set
    print('training......')
    tree.learn(training_set, attr_names, threshold, method='GiniGain')

    # Classify the test set using the tree we just constructed
    print('testing......')
    results = []
    for instance in test_set:
        result = tree.classify( instance[:-1] )
        results.append( result == instance[-1])

    # Accuracy
    accuracy = float(results.count(True))/float(len(results))
    print("accuracy: %.4f" % accuracy)
    if visualize:
        tree.visualize()
    
    return accuracy

In [8]:
run_decision_tree(threshold=500, visualize=True)

Number of records: 4898
training......
testing......
accuracy: 0.8037


0.803680981595092